In [ ]:
# !pip install diffusers[training]

In [ ]:
import os
from pathlib import Path

from dataclasses import dataclass
from datasets import load_dataset

from PIL import Image
import torch
import torch.nn.functional as F

from diffusers import UNet2DModel, DDPMScheduler, DDPMPipeline
from diffusers.optimization import get_cosine_schedule_with_warmup
from accelerate import Accelerator
from tqdm.auto import tqdm

## Training configuration

In [ ]:
@dataclass
class TrainingConfig:
    image_size = 32  # the generated image resolution
    
    train_batch_size = 16
    eval_batch_size = 16
    
    num_epochs = 15
    save_image_epochs = 1
    save_model_epochs = 1
    start_from_checkpoint = True
    
    gradient_accumulation_steps = 1
    learning_rate = 1e-4
    lr_warmup_steps = 500
    
    mixed_precision = "fp16"  # `fp16` for automatic mixed precision
    output_dir = "CIFAR10_Noise"  # Dir to save model and eval sample
    seed = 5


config = TrainingConfig()

## Load the dataset

In [ ]:
# Downloads Dataset from HuggingFace Repository
# config.dataset_name = "flwrlabs/celeba" # CelebA dataset
config.dataset_name = "uoft-cs/cifar10" # CIFAR10 dataset
# config.dataset_name = "ylecun/mnist" # MNIST dataset

# Load a subset of the dataset
dataset = load_dataset(config.dataset_name, split="train")
# dataset = dataset.select(range(20_000))

## Verify Dataset by Sampling

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 4, figsize=(16, 4))

'''CIFAR10'''
for i, image in enumerate(dataset[:4]["img"]):
    axs[i].imshow(image)
    axs[i].set_axis_off()
fig.show()

'''CelebA / MNIST'''
# for i, image in enumerate(dataset[:4]["image"]):
#     axs[i].imshow(image)
#     axs[i].set_axis_off()
# fig.show()


In [ ]:
from torchvision import transforms

'''CelebA / CIFAR10'''
preprocess = transforms.Compose(
    [
        transforms.Resize((config.image_size, config.image_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)

'''MNIST'''
# preprocess = transforms.Compose(
#     [
#         transforms.Resize((config.image_size, config.image_size)),
#         transforms.ToTensor(),
#         transforms.Normalize([0.5], [0.5]),
#     ]
# )

In [ ]:
'''CIFAR10'''
def transform(examples):
    images = [preprocess(image.convert("RGB")) for image in examples["img"]]
    return {"images": images}

'''CelebA / MNIST'''
# def transform(examples):
#     images = [preprocess(image.convert("RGB")) for image in examples["image"]]
#     return {"images": images}


dataset.set_transform(transform)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=config.train_batch_size, shuffle=True)

## (Optional) Load Pretrained Model

In [ ]:
checkpoint_number = None

if (checkpoint_number is not None):
  model = UNet2DModel.from_pretrained(os.path.join(config.output_dir, f"checkpoint_after_epoch_{checkpoint_number}", "unet"))
  noise_scheduler = DDPMScheduler.from_pretrained(os.path.join(config.output_dir, f"checkpoint_after_epoch_{checkpoint_number}", "scheduler"))

## Create a UNet Model and Noise Scheduler


In [ ]:
if (model is not None):
  '''MNIST'''
  # model = UNet2DModel(
  #     sample_size=config.image_size,
  #     in_channels=1,   # grayscale input
  #     out_channels=1,  # grayscale output
  #     layers_per_block=2,  # 2 ResNet layers per block for sufficient capacity
  #     block_out_channels=(32, 64, 128),  # Simpler channel progression for a smaller image
  #     down_block_types=(
  #         "DownBlock2D",      # 32×32 → 16×16
  #         "DownBlock2D",      # 16×16 → 8×8
  #         "AttnDownBlock2D",  # 8×8 → 4×4 with attention
  #     ),
  #     up_block_types=(
  #         "AttnUpBlock2D",  # 4×4 → 8×8 with attention
  #         "UpBlock2D",      # 8×8 → 16×16
  #         "UpBlock2D",      # 16×16 → 32×32
  #     ),
  # )

  '''CIFAR10'''
  model = UNet2DModel(
      sample_size=config.image_size,
      in_channels=3,  # the number of input channels, 3 for RGB images
      out_channels=3,  # the number of output channels
      layers_per_block=2,  # how many ResNet layers to use per UNet block
      block_out_channels=(128, 256, 256,256),  # the number of output channels for each UNet block
      down_block_types=(
          "DownBlock2D",      # a regular ResNet downsampling block
          "DownBlock2D",
          "DownBlock2D",
          "AttnDownBlock2D",  # a ResNet downsampling block with spatial self-attention
      ),
      up_block_types=(
          "AttnUpBlock2D",  # a ResNet upsampling block with spatial self-attention
          "UpBlock2D",      # a regular ResNet upsampling block
          "UpBlock2D",
          "UpBlock2D",
      ),
  )

  '''CelebA'''
  # model = UNet2DModel(
  #     sample_size=config.image_size,
  #     in_channels=3,  # the number of input channels, 3 for RGB images
  #     out_channels=3,  # the number of output channels
  #     layers_per_block=2,  # how many ResNet layers to use per UNet block
  #     block_out_channels=(128, 128, 256, 256, 512, 512),  # the number of output channels for each UNet block
  #     down_block_types=(
  #         "DownBlock2D",  # a regular ResNet downsampling block
  #         "DownBlock2D",
  #         "DownBlock2D",
  #         "DownBlock2D",
  #         "AttnDownBlock2D",  # a ResNet downsampling block with spatial self-attention
  #         "DownBlock2D",
  #     ),
  #     up_block_types=(
  #         "UpBlock2D",  # a regular ResNet upsampling block
  #         "AttnUpBlock2D",  # a ResNet upsampling block with spatial self-attention
  #         "UpBlock2D",
  #         "UpBlock2D",
  #         "UpBlock2D",
  #         "UpBlock2D",
  #     ),
  # )

# For noise prediction, use prediction_type="epsilon"
# For clean image prediction, use prediction_type="sample"
if (noise_scheduler is not None):
  noise_scheduler = DDPMScheduler(num_train_timesteps=1000, prediction_type="epsilon")

## Training Loop

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=config.lr_warmup_steps,
    num_training_steps=(len(train_dataloader) * config.num_epochs),
)

In [ ]:
#This will make grid of images
def make_grid(images, rows, cols):
    w, h = images[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    for i, image in enumerate(images):
        grid.paste(image, box=(i % cols * w, i // cols * h))
    return grid

#Perform evaluation
def evaluate(config, epoch, pipeline):
    # Sample some images from random noise (this is the backward diffusion process).
    # The default pipeline output type is `List[PIL.Image]`
    images = pipeline(
        batch_size=config.eval_batch_size,
        generator=torch.manual_seed(config.seed),
    ).images

    # Make a grid out of the images
    image_grid = make_grid(images, rows=4, cols=4)

    # Save the images
    test_dir = os.path.join(config.output_dir, "samples")
    os.makedirs(test_dir, exist_ok=True)
    image_grid.save(f"{test_dir}/{epoch:04d}.png")

In [ ]:
def train_loop(config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler):
    # Initialize accelerator and tensorboard logging
    accelerator = Accelerator(
        mixed_precision=config.mixed_precision,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        log_with="tensorboard",
        project_dir=os.path.join(config.output_dir, "logs"),
    )

    if accelerator.is_main_process:
        if config.output_dir is not None:
            os.makedirs(config.output_dir, exist_ok=True)
        accelerator.init_trackers("train_example")

    # Prepare everything
    # There is no specific order to remember, you just need to unpack the
    # objects in the same order you gave them to the prepare method.
    model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, lr_scheduler
    )

    global_step = 0

    # Training Loop
    for epoch in range(config.num_epochs):
        progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
        progress_bar.set_description(f"Epoch {epoch}")

        for step, batch in enumerate(train_dataloader):
            clean_images = batch["images"]

            # Sample noise to add to the images
            noise = torch.randn(clean_images.shape).to(clean_images.device)
            bs = clean_images.shape[0]

            # Sample a random timestep for each image
            timesteps = torch.randint(
                0, noise_scheduler.config.num_train_timesteps, (bs,), device=clean_images.device
            ).long()

            # Add noise to the clean images according to the noise magnitude at each timestep
            # (this is the forward diffusion process)
            noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)

            
            '''Noise Interpretation'''
            with accelerator.accumulate(model):
                noise_pred = model(noisy_images, timesteps, return_dict=False)[0]
                loss = F.mse_loss(noise_pred, noise)
                accelerator.backward(loss)

                accelerator.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()


            '''Clean Image Learning Interpretation'''
            # with accelerator.accumulate(model):
            #     # Predict the original image from noise
            #     clean_preds = model(noisy_images, timesteps, return_dict=False)[0]
            #     loss = F.mse_loss(clean_preds, clean_images)
            #     accelerator.backward(loss)

            #     accelerator.clip_grad_norm_(model.parameters(), 1.0)
            #     optimizer.step()
            #     lr_scheduler.step()
            #     optimizer.zero_grad()

            progress_bar.update(1)
            logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0], "step": global_step}
            progress_bar.set_postfix(**logs)
            accelerator.log(logs, step=global_step)
            global_step += 1

        # After each epoch you optionally sample some demo images with evaluate() and save the model
        if accelerator.is_main_process:
            pipeline = DDPMPipeline(unet=accelerator.unwrap_model(model), scheduler=noise_scheduler)

            if (epoch + 1) % config.save_image_epochs == 0 or epoch == config.num_epochs - 1:
                evaluate(config, epoch, pipeline)

            if (epoch + 1) % config.save_model_epochs == 0 or epoch == config.num_epochs - 1:
                pipeline.save_pretrained(os.path.join(config.output_dir, f"checkpoint_after_epoch_{epoch}"))


# Start Training

In [ ]:
from accelerate import notebook_launcher

args = (config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler)
notebook_launcher(train_loop, args, num_processes=1)